In [ ]:
import requests, json, os, datetime
from bs4 import BeautifulSoup

In [ ]:
def get_westpac_pdfs():
    FAU_REGEX = re.compile('.+?Fortnightly-Agri-Update-(\d+)-(\S+)-(\d+)')
    WESTPAC_URL = 'https://www.westpac.co.nz/agribusiness/agri-information/' + \
                  'fortnightly-agri-updates/'
    response = requests.get(WESTPAC_URL)
    soup = BeautifulSoup(response.text, 'html.parser')
    for link in soup.find_all('a'):
        link_url = link.get('href')
        if link_url == None:
            continue
        match = FAU_REGEX.match(link_url)
        if match != None:
            date = match[3]+ '-' + match[2] + '-' + match[1]
            output_file = '../data/forecasts/westpac/' + date + '.pdf'
            if not os.path.isfile(output_file):
                response = requests.get('https://www.westpac.co.nz/' + link_url)
                with open(output_file, 'wb') as file:
                    file.write(response.content)
        
get_westpac_pdfs()

In [ ]:
def run_agri_hq_scraper():
    response = requests.get('https://dairy-tools.nzx.com/fgmp/calculator_widget.json',
                            data={'client_key': os.environ['agrihq_client_key']})
    today = datetime.datetime.now().strftime('%Y-%m-%d')
    with open('../data/forecasts/agrihq/' + today + '.json', 'w') as file:
        json.dump(response.json()['calculations'], file, indent=2)
        
run_agri_hq_scraper()